# Computational Phase Retrieval with Tensor Methods

## Device Information

In [1]:
!nvidia-smi

Wed Jul 14 16:18:24 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.80       Driver Version: 460.80       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 165...  Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   41C    P8     4W /  N/A |    312MiB /  3911MiB |     10%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Import Required Libraries

In [2]:
import tensorflow as tf
print(f"Tensorflow version: {tf.__version__}")

import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')

if gpus:
    try:
        print("Num GPUs Available: ", len(gpus))
        for gpu in gpus:
            # Allow memory growth for the GPU.
            # Reference: https://www.tensorflow.org/guide/gpu
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized.
        print(e)

from matplotlib import pyplot as plt
plt.style.use('dark_background')
import numpy as np
import tensorly as tl
import cv2
import time
import os
from PIL import Image

import matplotlib.cm as cm
import seaborn as sns

from matplotlib import rc, rcParams
rcParams['font.family'] = 'Cubano'
rc('text', usetex=False)

import warnings

Tensorflow version: 2.5.0
Num GPUs Available:  1
1 Physical GPUs, 1 Logical GPUs


## GPU Benchmark

Run preliminarily to avoid cold-start.

Reference: https://www.tensorflow.org/guide/gpu

In [3]:
tf.debugging.set_log_device_placement(True)

n = 500
num_iters = 10

'''
Test with TensorFlow GPU.
'''
start_tf = time.time()

for i in range(num_iters):
    # Tensors are defaultly placed on the GPU (CPU would be considerably slower due
    # to the incurred communication cost).
    # with tf.device('/CPU:0'):
    a = tf.ones((n, n))
    b = tf.ones((n, n))

    # Run on the GPU
    c = tf.matmul(a, b)

print(f'Elapsed time with TensorFlow GPU: {time.time() - start_tf}')

'''
Test with Numpy.
'''
start_np = time.time()

# for i in range(num_iters):
#     a = np.ones((n, n))
#     b = np.ones((n, n))

#     c = np.dot(a, b)

print(f'Elapsed time with Numpy: {time.time() - start_np}') # CAN BE SLOW


Elapsed time with TensorFlow GPU: 0.5779464244842529
Elapsed time with Numpy: 3.266334533691406e-05


## Low Rank Phase Retrieval

References:

\[1\] Namrata Vaswani, Seyedehsara Nayer, Yonina C. Eldar. *Low Rank Phase Retrieval*. https://rutgers.box.com/s/dntl0sh157p62rgi1zerdaxrqthugr32

\[2\] Namrata Vaswani. *Nonconvex Structured Phase Retrieval*. https://rutgers.box.com/s/x02w8frd1ep01cxdjlnojufa9npvstsz.

\[3\] Tamara G. Kolda, Brett W. Bader. *Tensor Decompositions and Applications*. https://rutgers.box.com/s/aq9psx3mgwhms6rrzlhn94h56c3oshox. 




### Define Data Directories

In [4]:
INPUT_DIR = './videos/' # directory of the test videos
OUTPUT_DIR = './output/' # output directory
FRAMES_DIR = './ouput/frames/' # output directory of the extracted video frames 

### Load the Test Video

In [7]:
# Read the video.
video_path = INPUT_DIR + os.listdir(INPUT_DIR)[0] # define video path
cap = cv2.VideoCapture(video_path) # read the video from path
video_name = os.listdir(INPUT_DIR)[0].split('.')[0] # get the name of the video

# Creat the folder to store the extracted frames of the video.
try:
    if not os.path.exists(FRAMES_DIR + video_name):
        os.makedirs(FRAMES_DIR + video_name)
    else:
        print('Directory already exists!')
except OSError:
    print('OS ERROR')

k = 0 # frame number, k = 0, 1, 2, ..., q - 1
Xlist = []
Rhat = 0
while (True):
    # Capture the video frame-by-frame.
    # Code adopted: https://docs.opencv.org/3.4/dd/d43
    # tutorial_py_video_display.html
    ret, frame = cap.read()

    # If the frame is read correctly the return boolean (ret) is true.
    if not ret:
        print("Cannot receive frame (probably end of stream). Exiting...")
        break
    else:
        # Convert the frame to grayscale.
        gray_frame_original = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        scale = 0.025
        width = int(gray_frame_original.shape[1] * scale)
        height = int(gray_frame_original.shape[0] * scale)
        gray_frame = cv2.resize(gray_frame_original, (width, height))
        name = FRAMES_DIR + video_name + '/frame-' + str(k) + '.jpg'
        print('DEBUG: Captured...' + name)
        svds = np.linalg.svd(gray_frame)[1]
        max_svd, min_svd = np.max(svds), np.min(svds)
        normalized_svds = svds / (max_svd - min_svd)
        Rhat += np.sum(normalized_svds > 0.1)
        cv2.imwrite(name, gray_frame)
        # plt.plot(range(480), normalized_svds)
        # plt.show()
        Xlist.append(gray_frame)

        k += 1
Rhat = Rhat // k + 1
print('Approximate rank of each frame: ', Rhat)

# Release the capture when finished.
cap.release()
cv2.destroyAllWindows()

Directory already exists!
DEBUG: Captured..../ouput/frames/sara/frame-0.jpg
DEBUG: Captured..../ouput/frames/sara/frame-1.jpg
DEBUG: Captured..../ouput/frames/sara/frame-2.jpg
DEBUG: Captured..../ouput/frames/sara/frame-3.jpg
DEBUG: Captured..../ouput/frames/sara/frame-4.jpg
DEBUG: Captured..../ouput/frames/sara/frame-5.jpg
DEBUG: Captured..../ouput/frames/sara/frame-6.jpg
DEBUG: Captured..../ouput/frames/sara/frame-7.jpg
DEBUG: Captured..../ouput/frames/sara/frame-8.jpg
DEBUG: Captured..../ouput/frames/sara/frame-9.jpg
DEBUG: Captured..../ouput/frames/sara/frame-10.jpg
DEBUG: Captured..../ouput/frames/sara/frame-11.jpg
DEBUG: Captured..../ouput/frames/sara/frame-12.jpg
DEBUG: Captured..../ouput/frames/sara/frame-13.jpg
DEBUG: Captured..../ouput/frames/sara/frame-14.jpg
DEBUG: Captured..../ouput/frames/sara/frame-15.jpg
DEBUG: Captured..../ouput/frames/sara/frame-16.jpg
DEBUG: Captured..../ouput/frames/sara/frame-17.jpg
DEBUG: Captured..../ouput/frames/sara/frame-18.jpg
DEBUG: Captured

### Create the true signal tensor.

Tensors are multi-dimensional arrays with a uniform type (`dtype`). All tensors are immutable like Python numbers and strings: you can never update the contents of a tensor, only create a new one.

**Note**: In libraries like tensorflow, the rank of the tensor actually denotes the order of the tensor in our convention. We call the `rank` of a tensor in a similar manner as the rank of a matrix.

The gray-scaled signal is modeled as a three-ordered tensor $\boldsymbol{\mathcal{X}} \in \mathbb{R}^{I_1 \times I_2 \times q}$, where $I_1 \times I_2$ correspond to the pixel coordinates within each frame and $q$ is the total number of frames captured.

**Signal Dimension**

In [8]:
Xast = tf.constant(Xlist, tf.float32)
q, I1, I2 = Xast.shape
Xast = tf.reshape(Xast, [I1, I2, q])
print(f'The dimension of the true signal tensor: I1 x I2 x q: {I1} x {I2} x {q}')
print(f'Sample complexity for rank {Rhat}: O({(q + I1 + I2) * Rhat})')

The dimension of the true signal tensor: I1 x I2 x q: 12 x 16 x 105
Sample complexity for rank 3: O(399)


### Generate Phaseless Measurements

In [32]:
A = [] # measurement tensor in list format
m = 2000 # number of measurements

np.random.seed(5) # set random seed

# Generate i.i.d. measurement tensors.

for j in range(m):
    Aj = []
    for k in range(q):
        Ajk = np.random.randn(I1, I2) # i.i.d. normal measurement
        Aj.append(Ajk)
    A.append(Aj)

A = tf.reshape(tf.constant(A, tf.float32), [m, I1, I2, q])

In [33]:
# Generate phaseless measurements.
Y = np.zeros((m, q)) # matrix of the phaseless measurements

for j in range(m):
    for k in range(q):
        Y[j, k] = tf.tensordot(A[j,:,:,k], Xast[:,:,k], axes=([0, 1], [0, 1]))**2

Y = tf.cast(Y, tf.float32)

In [34]:
print('DEBUG')
print(f'\nDimension of the phaseless measurement matrix: m x q: {Y.shape[0]} x {Y.shape[1]}\n')
print('Phaseless Measurements:\n\n', Y)

DEBUG

Dimension of the phaseless measurement matrix: m x q: 2000 x 105

Phaseless Measurements:

 tf.Tensor(
[[2.5711866e+05 3.1022844e+05 3.9392282e+06 ... 2.3886342e+06
  2.2918038e+05 8.5250575e+05]
 [9.2204369e+05 3.8461139e+02 1.7245771e+06 ... 1.0882722e+06
  5.4024975e+05 2.4865598e+04]
 [1.8679306e+05 1.6129268e+07 2.0637980e+06 ... 9.3982625e+05
  9.7934941e+03 6.9455894e+05]
 ...
 [3.3589328e+05 2.4439505e+06 9.8705138e+05 ... 4.2333256e+05
  1.4663504e+06 2.0064834e+06]
 [2.2953018e+06 1.5385001e+06 4.7616666e+05 ... 6.0902080e+06
  1.2677964e+07 5.5812488e+05]
 [1.0500918e+06 1.9997820e+06 2.2203017e+05 ... 7.7996925e+06
  1.5764875e+06 6.1906905e+06]], shape=(2000, 105), dtype=float32)


In [35]:
def initialize(I1, I2, q, R):
    """Initialize factor matrices."""

    U1 = tf.cast(np.random.randn(I1, R), tf.float32)
    U2 = tf.cast(np.random.randn(I2, R), tf.float32)
    U = [U1,U2]
    B = tf.Variable(np.random.randn(q, R), tf.float32)

    return U, B

In [36]:
def kruskal(U, B, R, Lambda=None, type='CP'):
    """Construct Tensor from Kruskal Formulation.

        Args:
            U: list consisting of two factor matrices U1 (I1 x R)
                and U2 (I2 x R) for the three-way case.
            B: the B (q x R) factor matrix.
            R: assumped rank (a scalar) of the low-rank tensor.
            Lambda: normalization factors (length R).
        
        Returns:
            Xhat: signal estimate (I1 x I2 x q).
    """
    warnings.filterwarnings("ignore", category=RuntimeWarning)
    B = tf.cast(B, tf.float32)
    if type == 'CP':
        U1, U2 = U[0], U[1]
        I1, I2, q = U1.shape[0], U2.shape[0], B.shape[0]
        Xhat = tf.zeros([I1, I2, q])
        if Lambda is None:
            Lambda = tf.ones([R,])
        for r in range(R):
            U1U2 = tf.tensordot(U1[:, r], U2[:, r], axes=0)
            Xhat += Lambda[r] * tf.tensordot(U1U2, B[:, r], axes=0)
        
        return Xhat
    else:
        return None

In [37]:
def test_kruskal():
    from tensorly.decomposition import parafac

    X = tl.tensor(np.arange(24).reshape((3, 4, 2)), dtype=tl.float32)
    weights, factors = parafac(X, rank=3)

    X_test = kruskal([factors[0], factors[1]], factors[2], 3, weights)

    err = X - X_test

    print(f'DEBUG | Mean squared error for reconstruction: {tf.math.reduce_sum(tf.multiply(err, err)).numpy()}')

test_kruskal() # test Kruskal tensor constructor



DEBUG | Mean squared error for reconstruction: 3.952038696297677e-11


In [38]:
def reconstruct_error(Xast, U, B, R=10):
    err = Xast - kruskal(U, B, R)

    return tf.math.reduce_sum(tf.multiply(err, err)).numpy()

In [39]:
def descent(Uhat, Bhat, A, Y, R, max_iter):
    U1, U2 = Uhat[0], Uhat[1]
    I1, I2 = U1.shape[0], U2.shape[0]
    m = A.shape[0]
    q = Bhat.shape[0]
    Bhat = Bhat.numpy()

    for t in range(max_iter):
        print(f'Iteration-{t}')
        print('Computing....')
        Xhat = kruskal(Uhat, Bhat, R)
        Cy = np.zeros([m, q])
        # Bhat = tf.Variable(Bhat, tf.float32)

        '''
        Update Phase.
        '''
        for k in range(q):
            AX = []
            Xk = Xhat[:,:,k]
            for j in range(m):
                AX.append(
                    tf.tensordot(A[j,:,:,k], Xk, axes=([0, 1], [0, 1])))
            Ck = tf.linalg.diag(tf.math.angle(AX))
            Cy[:, k] = tf.tensordot(Ck, tf.math.sqrt(Y[:,k]), axes=1)

        '''
        Solve for U1.
        '''
        matrix = np.zeros((I1,R))
        rhs = 0
        for k in range(q):
            bk = Bhat[k,:]
            bU2_kr =  tf.multiply(U2, bk) # khatri-rao product
            for j in range(m):
                A1 = tl.unfold(A[j,:,:,k], 0)
                rhs += Cy[j, k]
                matrix += tf.linalg.matmul(A1, bU2_kr)
        
        matrix = tf.cast(tf.transpose(matrix), tf.float32)
        rhs = tf.cast(rhs, tf.float32)
        U1 = tf.linalg.lstsq(matrix, tf.linalg.diag(tf.fill((R,), rhs)))

        '''
        Solve for U2.
        '''
        matrix = np.zeros((I2,R))
        rhs = 0
        for k in range(q):
            bk = Bhat[k,:]
            bU1_kr =  tf.multiply(U1, bk) # khatri-rao product
            for j in range(m):
                A2 = tl.unfold(A[j,:,:,k], 1)
                rhs += Cy[j, k]
                matrix += tf.linalg.matmul(A2, bU1_kr)
        
        matrix = tf.cast(tf.transpose(matrix), tf.float32)
        rhs = tf.cast(rhs, tf.float32)
        U2 = tf.linalg.lstsq(matrix, tf.linalg.diag(tf.fill((R,), rhs)))

        '''
        Solve for bk's.
        '''
        vector = np.zeros((R,))
        rhs = 0
        for k in range(q):
            bk = Bhat[k,:]
            U2U1_kr =  tl.tenalg.khatri_rao([U2, U1]) # khatri-rao product
            for j in range(m):
                A3 = tf.reshape(A[j,:,:,k], [I1 * I2,])
                rhs += Cy[j, k]
                vector += tf.linalg.matvec(tf.transpose(U2U1_kr), A3)
            vector = tf.cast(vector, tf.float32)
            rhs = tf.cast(rhs, tf.float32)
            Bhat[k,:] = tf.multiply(vector, 1 / rhs)
        
        # print(
        #     f'Reconstruction error: {reconstruct_error([U1, U2], Bhat, R):.2f}.')
    
    Uhat = [U1, U2]
    
    return Uhat, Bhat

In [40]:
def plrpr(A, Y, R=5, max_iter=1):
    """Polyadic Low Rank Phase Retrieval.
    """
    Uinit, Binit = initialize(I1, I2, q, R)
    
    Uhat, Bhat = descent(Uinit, Binit, A, Y, R, max_iter)

    Xhat = kruskal(Uhat, Bhat, R)
    
    return Xhat

In [31]:
def test_plrpr(A, Y, Xast, R=10, max_iter=10):
    Xhat = plrpr(A, Y, R, max_iter)
    filename = FRAMES_DIR + video_name + '/frame-reconstructed-' + str(k) + '.jpg'
    cv2.imwrite(filename, Xhat[:,:,0].numpy())
    err = Xast - Xhat

    MSE = tf.math.reduce_sum(tf.multiply(err, err)).numpy()

    print(f'DEBUG | Mean squared error for reconstruction: {MSE}')

test_plrpr(A, Y, Xast, R = 20, max_iter = 5)


Iteration-0
Computing....
Iteration-1
Computing....
Iteration-2
Computing....
Iteration-3
Computing....
Iteration-4
Computing....
DEBUG | Mean squared error for reconstruction: 1495695097856.0


In [ ]:
class TensorLRPR

In [ ]:
class TensorUtils

- Initialization (Spectral, HOSVD) for CP formulation.
- Complex measurements.
